### CSV Prettifier

- Input: path of raw CSV files.
- Output: CSV (single sheet)/XLSX (multiple sheets) files with improved readability.

In [54]:
def CSV_prettifier(path, output):
    import pandas as pd
    import numpy as np
    import seaborn as sns
    import os
    import glob
    
    all_files = glob.glob1(path,"*csv")
    
    if output == 'csv':
        
        li = []

        for filename in all_files:
            x = path+ '/' +filename
            df = pd.read_csv(x, index_col=None, header=0, sep='~~', engine='python')
            df.set_index('Report Suite ID    ', inplace=True)
            li.append(df)

        frame = pd.concat(li, axis=1, sort='False')
        frame.to_csv('ShawQA-samesheet.csv')
        
    elif output == 'xlsx':
        di = {}

        for filename in all_files:
            x = path+ '/' +filename
            df = pd.read_csv(x, index_col=None, header=0, sep='~~', engine='python')
            df.set_index('Report Suite ID    ', inplace=True)
            di.update({ filename : df})
    
        writer = pd.ExcelWriter('ShawQA-sheets.xlsx', engine='xlsxwriter')

        for sheet in di.keys():
            di[sheet].to_excel(writer, sheet_name=sheet, index=True)
    
        writer.save()

In [55]:
path = '/Users/adekadam/Documents/GitHub/shaw-data-bot/hits'